In [19]:
# Notwendige Bibliotheken importieren
import pandas as pd
import requests
import random
import datetime

## Laden des Datensatzes

Jetzt laden wir unseren Datensatz. Wir gehen davon aus, dass der Datensatz eine CSV-Datei mit Verkaufsdaten enthält.

Fetch Public and public Parking Data from the APIs

In [20]:
# API URL for Freiburg
api_url_freiburg = "https://api.parkendd.de/Freiburg"

# Fetch the data from the WFS API
response_freiburg = requests.get(api_url_freiburg)

if response_freiburg.status_code == 200:
    public_parking_data = response_freiburg.json()

    # Extract relevant data
    parking_data_public = []
    for site in public_parking_data['lots']:

        if site.get("purpose") and site.get("purpose") != "CAR":
            continue
        
        coords = site.get("coords", {})
        parking_data_public.append({
            "public_id": site.get("id"),
            "name": site.get("name"),
            "address": site.get("address"),
            "latitude": coords.get("lat"),
            "longitude": coords.get("lng"),
            "price_per_hour": random.choice([3.80, 3.20, 1.60]),
            "capacity": site.get("total"),
            "available_space": site.get("free"),
            "opening_time": random.choice(["Weekdays 9AM-11PM", "Weekdays 9AM-7PM", "Weekdays 6AM-6PM", "Monday-Sunday 9AM-11PM"])
        })
    
    # Convert to DataFrame
    df_freiburg = pd.DataFrame(parking_data_public)
    print("Public Parking Data:", df_freiburg.head())
else:
    print("Failed to fetch MobiData BW parking data.")

Public Parking Data:                    public_id                name address   latitude  \
0  freiburgzentrumoberwiehre  Zentrum Oberwiehre     P21  47.987991   
1         freiburgggrafhalle       G.-Graf-Halle      P6  47.999300   
2           freiburgkarlsbau            Karlsbau     P13  47.997446   
3     freiburgbahnhofsgarage      Bahnhofsgarage      P1  47.996932   
4    freiburgschwarzwaldcity    Schwarzwald City      P9  47.997206   

   longitude  price_per_hour  capacity  available_space       opening_time  
0   7.871473             1.6       274              267  Weekdays 9AM-11PM  
1   7.846800             1.6        56               50  Weekdays 9AM-11PM  
2   7.853848             3.2       977              838  Weekdays 9AM-11PM  
3   7.841420             3.2         0                0  Weekdays 9AM-11PM  
4   7.851119             3.2       416                0   Weekdays 9AM-7PM  


In [21]:
# URL for MobiData BW API (public parking data)
api_url_gebündelte_parkplätze_parkbauten = "https://api.mobidata-bw.de/park-api/api/public/v3/parking-sites"

# Fetch data from MobiData BW API
response_gebündelte_parkplätze_parkbauten = requests.get(api_url_gebündelte_parkplätze_parkbauten)

# Check the response and parse the JSON data
if response_gebündelte_parkplätze_parkbauten.status_code == 200:
    public_parking_data = response_gebündelte_parkplätze_parkbauten.json()
    
    # Extract relevant data (you can modify based on the actual response structure)
    parking_data_public = []
    for site in public_parking_data['items']:
        
        if site.get("purpose") and site.get("purpose") != "CAR":
            continue

        latitude = float(site.get("lat", 0)) 
        longitude = float(site.get("lon", 0))  
        
        if not (47.52 <= latitude <= 49.79 and 7.50 <= longitude <= 10.49):
            continue
        
        parking_data_public.append({
            "public_id": site.get("id"),
            "name": site.get("name"),
            "address": site.get("address"),
            "latitude": latitude,
            "longitude": longitude,
            "price_per_hour": random.choice([3.80, 3.20, 1.60]),
            "capacity": site.get("capacity"),
            "available_space": site.get("realtime_free_capacity"),
            "opening_time": random.choice(["Weekdays 9AM-11PM", "Weekdays 9AM-7PM", "Weekdays 6AM-6PM", "Monday-Sunday 9AM-11PM"])
        })
    
    # Convert to DataFrame
    df_public_parking = pd.DataFrame(parking_data_public)
    print("Public Parking Data:", df_public_parking.head())
else:
    print("Failed to fetch MobiData BW parking data.")


Public Parking Data:    public_id                         name  \
0        887  Listplatz 1 / Bahnhofstraße   
1        888          Obere Wässere 3 - 7   
2      17151                    Parkplatz   
3        757                Bahnhofstraße   
4        758                Am Südbahnhof   

                                   address   latitude  longitude  \
0  Listplatz 1 / Bahnhofstraße, Reutlingen  48.495983   9.210331   
1          Obere Wässere 3 - 7, Reutlingen  48.488955   9.216701   
2                                Stuttgart  48.834065   9.152340   
3                Bahnhofstraße, Reutlingen  48.497575   9.211077   
4                Am Südbahnhof, Reutlingen  48.482762   9.229821   

   price_per_hour  capacity  available_space            opening_time  
0             1.6     198.0              NaN        Weekdays 6AM-6PM  
1             3.8     181.0              NaN        Weekdays 6AM-6PM  
2             1.6     200.0              NaN  Monday-Sunday 9AM-11PM  
3             3

In [22]:
df_freiburg

,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time
0,freiburgzentrumoberwiehre,Zentrum Oberwiehre,P21,47.987991,7.871473,1.6,274,267,Weekdays 9AM-11PM
1,freiburgggrafhalle,G.-Graf-Halle,P6,47.999300,7.846800,1.6,56,50,Weekdays 9AM-11PM
2,freiburgkarlsbau,Karlsbau,P13,47.997446,7.853848,3.2,977,838,Weekdays 9AM-11PM
3,freiburgbahnhofsgarage,Bahnhofsgarage,P1,47.996932,7.841420,3.2,0,0,Weekdays 9AM-11PM
4,freiburgschwarzwaldcity,Schwarzwald City,P9,47.997206,7.851119,3.2,416,0,Weekdays 9AM-7PM
5,freiburgschwabentor,Schwabentor,P16,47.991264,7.856310,1.6,221,176,Weekdays 9AM-7PM
6,freiburgunterlinden,Unterlinden,P8,47.997868,7.848868,3.8,218,200,Monday-Sunday 9AM-11PM
7,freiburgkonzerthaus,Konzerthaus,P2,47.995290,7.840836,3.2,421,337,Weekdays 6AM-6PM
8,freiburgrotteck,Rotteck,P10,47.995801,7.846298,3.2,300,253,Weekdays 6AM-6PM
9,freiburgpaedhochschule,Päd. Hochschule,P20,47.982361,7.890891,1.6,168,37,Weekdays 6AM-6PM


In [23]:
df_public_parking

,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time
0,887,Listplatz 1 / Bahnhofstraße,"Listplatz 1 / Bahnhofstraße, Reutlingen",48.495983,9.210331,1.6,198.0,NaN,Weekdays 6AM-6PM
1,888,Obere Wässere 3 - 7,"Obere Wässere 3 - 7, Reutlingen",48.488955,9.216701,3.8,181.0,NaN,Weekdays 6AM-6PM
2,17151,Parkplatz,Stuttgart,48.834065,9.152340,1.6,200.0,NaN,Monday-Sunday 9AM-11PM
3,757,Bahnhofstraße,"Bahnhofstraße, Reutlingen",48.497575,9.211077,3.8,128.0,NaN,Weekdays 6AM-6PM
4,758,Am Südbahnhof,"Am Südbahnhof, Reutlingen",48.482762,9.229821,1.6,NaN,NaN,Weekdays 9AM-11PM
...,...,...,...,...,...,...,...,...,...
3339,392,"Parkgarage Landesoberkasse (LOK), Steinhäusers...","Steinhäuserstraße 11 - Zufahrt Holtzstraße 4A,...",49.002434,8.382499,3.2,97.0,78.0,Monday-Sunday 9AM-11PM
3340,438,"Parkhaus Pädagogische Hochschule (PH), Kunzenweg","Kunzenweg, 79117 Freiburg",47.982064,7.891118,3.8,350.0,37.0,Monday-Sunday 9AM-11PM
3341,17426,Parkplatz,Eglosheim,48.909889,9.181621,1.6,140.0,NaN,Weekdays 9AM-11PM
3342,17439,Parkplatz,Marbach (Neckar),48.943490,9.262973,1.6,67.0,NaN,Monday-Sunday 9AM-11PM


Data Cleaning

In [24]:
# Clean the Freiburg parking data
df_freiburg_cleaned = df_freiburg.dropna(subset=["public_id", "name", "address", "latitude", "longitude", "price_per_hour", "capacity"])  

# Clean the public parking data
df_public_parking_cleaned = df_public_parking.dropna(subset=["public_id", "name", "address", "latitude", "longitude", "price_per_hour", "capacity"]) 

# Function to fill NaN in 'available_space' with half of 'capacity'
def fill_available_space_with_half_capacity(df):
    df['available_space'] = df['available_space'].fillna(df['capacity'] / 2)
    df['available_space'] = df['available_space'].astype(int)  # Ensure integer values
    return df

# Apply the function to both datasets
df_freiburg_cleaned = fill_available_space_with_half_capacity(df_freiburg_cleaned)
df_public_parking_cleaned = fill_available_space_with_half_capacity(df_public_parking_cleaned)

# Remove duplicates based on parking ID
df_freiburg_cleaned = df_freiburg_cleaned.drop_duplicates(subset=["public_id"])
df_public_parking_cleaned = df_public_parking_cleaned.drop_duplicates(subset=["public_id"])

df_public_parking_cleaned


C:\Users\vae.tiolamon\AppData\Local\Temp\ipykernel_18340\3302152171.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['available_space'] = df['available_space'].fillna(df['capacity'] / 2)
C:\Users\vae.tiolamon\AppData\Local\Temp\ipykernel_18340\3302152171.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['available_space'] = df['available_space'].astype(int)  # Ensure integer values


,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time
0,887,Listplatz 1 / Bahnhofstraße,"Listplatz 1 / Bahnhofstraße, Reutlingen",48.495983,9.210331,1.6,198.0,99,Weekdays 6AM-6PM
1,888,Obere Wässere 3 - 7,"Obere Wässere 3 - 7, Reutlingen",48.488955,9.216701,3.8,181.0,90,Weekdays 6AM-6PM
2,17151,Parkplatz,Stuttgart,48.834065,9.152340,1.6,200.0,100,Monday-Sunday 9AM-11PM
3,757,Bahnhofstraße,"Bahnhofstraße, Reutlingen",48.497575,9.211077,3.8,128.0,64,Weekdays 6AM-6PM
5,759,Tannenberger Straße,"Tannenberger Straße, Reutlingen",48.511669,9.204728,3.8,46.0,23,Weekdays 6AM-6PM
...,...,...,...,...,...,...,...,...,...
3339,392,"Parkgarage Landesoberkasse (LOK), Steinhäusers...","Steinhäuserstraße 11 - Zufahrt Holtzstraße 4A,...",49.002434,8.382499,3.2,97.0,78,Monday-Sunday 9AM-11PM
3340,438,"Parkhaus Pädagogische Hochschule (PH), Kunzenweg","Kunzenweg, 79117 Freiburg",47.982064,7.891118,3.8,350.0,37,Monday-Sunday 9AM-11PM
3341,17426,Parkplatz,Eglosheim,48.909889,9.181621,1.6,140.0,70,Weekdays 9AM-11PM
3342,17439,Parkplatz,Marbach (Neckar),48.943490,9.262973,1.6,67.0,33,Monday-Sunday 9AM-11PM


Data Integration

In [25]:
# Ensure latitude and longitude columns are numeric and consistent
df_freiburg_cleaned["latitude"] = pd.to_numeric(df_freiburg_cleaned["latitude"], errors="coerce")
df_freiburg_cleaned["longitude"] = pd.to_numeric(df_freiburg_cleaned["longitude"], errors="coerce")

df_public_parking_cleaned["latitude"] = pd.to_numeric(df_public_parking_cleaned["latitude"], errors="coerce")
df_public_parking_cleaned["longitude"] = pd.to_numeric(df_public_parking_cleaned["longitude"], errors="coerce")

# Drop rows with NaN in latitude or longitude
df_freiburg_cleaned = df_freiburg_cleaned.dropna(subset=["latitude", "longitude"])
df_public_parking_cleaned = df_public_parking_cleaned.dropna(subset=["latitude", "longitude"])

# Round latitude and longitude to the same precision for matching
df_freiburg_cleaned["latitude"] = df_freiburg_cleaned["latitude"].round(6)
df_freiburg_cleaned["longitude"] = df_freiburg_cleaned["longitude"].round(6)

df_public_parking_cleaned["latitude"] = df_public_parking_cleaned["latitude"].round(6)
df_public_parking_cleaned["longitude"] = df_public_parking_cleaned["longitude"].round(6)

# Merge the DataFrames, prioritizing df_public_parking_cleaned data
df_combined = pd.merge(
    df_freiburg_cleaned,
    df_public_parking_cleaned,
    on=["latitude", "longitude"],
    how="outer",
    suffixes=("_freiburg", "_public")
)

# Prioritize data from df_public_parking_cleaned
for column in df_public_parking_cleaned.columns:
    if column not in ["latitude", "longitude"]:  # Exclude keys used for merging
        df_combined[column] = df_combined[column + "_public"].combine_first(df_combined[column + "_freiburg"])

# Drop redundant columns
columns_to_drop = [col for col in df_combined.columns if col.endswith("_freiburg") or col.endswith("_public")]
df_combined = df_combined.drop(columns=columns_to_drop)

df_combined



,latitude,longitude,public_id,name,address,price_per_hour,capacity,available_space,opening_time
0,47.520513,8.585892,9352.0,Embrach-Rorbas,"Bahnstrasse 9, 8424 Embrach-Rorbas",3.8,28.0,14.0,Monday-Sunday 9AM-11PM
1,47.522619,7.689327,9354.0,Pratteln,"Güterstrasse 19, 4133 Pratteln",3.2,58.0,29.0,Weekdays 6AM-6PM
2,47.523924,8.537127,19807.0,Bülach,"Schaffhauserstrasse 100, 8180 Bülach",1.6,234.0,117.0,Weekdays 6AM-6PM
3,47.524211,9.202017,8993.0,Kradolf,"Hauptstrasse 57, 9214 Kradolf",3.2,16.0,8.0,Monday-Sunday 9AM-11PM
4,47.525776,8.775885,9116.0,Wiesendangen,"Alte Frauenfelderstrasse 3, 8542 Wiesendangen",1.6,35.0,17.0,Weekdays 6AM-6PM
...,...,...,...,...,...,...,...,...,...
2531,49.762093,9.512663,17773.0,Parkplatz,Wertheim (TBB),3.2,9.0,4.0,Weekdays 6AM-6PM
2532,49.762799,9.512175,17771.0,Parkplatz,Wertheim (TBB),1.6,3.0,1.0,Weekdays 9AM-7PM
2533,49.785998,9.501370,17777.0,Parkplatz,Bestenheid,1.6,14.0,7.0,Weekdays 6AM-6PM
2534,49.786031,9.500884,17776.0,Parkplatz,Bestenheid,3.8,11.0,5.0,Weekdays 6AM-6PM


In [26]:
# Define desired column order
desired_columns = [
    "public_id", "name", "address", "latitude", "longitude", "price_per_hour", "capacity", "available_space", "opening_time"
]

# Reorder the columns in the combined DataFrame
df_combined = df_combined[[col for col in desired_columns if col in df_combined.columns]]

df_combined['public_id'] = df_combined['public_id'].apply(lambda x: f"PUB{random.randint(10000, 99999)}")

df_combined

,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time
0,PUB26729,Embrach-Rorbas,"Bahnstrasse 9, 8424 Embrach-Rorbas",47.520513,8.585892,3.8,28.0,14.0,Monday-Sunday 9AM-11PM
1,PUB49246,Pratteln,"Güterstrasse 19, 4133 Pratteln",47.522619,7.689327,3.2,58.0,29.0,Weekdays 6AM-6PM
2,PUB46162,Bülach,"Schaffhauserstrasse 100, 8180 Bülach",47.523924,8.537127,1.6,234.0,117.0,Weekdays 6AM-6PM
3,PUB12434,Kradolf,"Hauptstrasse 57, 9214 Kradolf",47.524211,9.202017,3.2,16.0,8.0,Monday-Sunday 9AM-11PM
4,PUB30278,Wiesendangen,"Alte Frauenfelderstrasse 3, 8542 Wiesendangen",47.525776,8.775885,1.6,35.0,17.0,Weekdays 6AM-6PM
...,...,...,...,...,...,...,...,...,...
2531,PUB44047,Parkplatz,Wertheim (TBB),49.762093,9.512663,3.2,9.0,4.0,Weekdays 6AM-6PM
2532,PUB57780,Parkplatz,Wertheim (TBB),49.762799,9.512175,1.6,3.0,1.0,Weekdays 9AM-7PM
2533,PUB24429,Parkplatz,Bestenheid,49.785998,9.501370,1.6,14.0,7.0,Weekdays 6AM-6PM
2534,PUB20328,Parkplatz,Bestenheid,49.786031,9.500884,3.8,11.0,5.0,Weekdays 6AM-6PM


In [27]:
df_combined.to_json('public_parking.json', orient='records', indent=4)

print("DataFrame has been saved to 'public_parking.json'")

DataFrame has been saved to 'public_parking.json'
